# Лабораторная работа № 4

## Постановка задачи

### Часть 1 
- Реализовать алгоритм базовой модели Идена; 
- Визуализировать рост кластера; 
- В лог-лог масштабе построить зависимости $N(R_{гир})$ и $N_s(R_{гир})$; 
- Определить размерность $d$ и $d_s$.

### Часть 2
- Реализовать алгоритм модели экранированного роста;
- Определить размерность кластера $d$ для случаев: $\psi < 2$, $\psi = 2$ и $\psi > 2$.

## Часть 1

### Реализация алгоритма базовой модели Идена

#### Описание

Рост кластера имитируется последовательным заполнением узлов квадратной решетки по периметру кластера.

В начальный момент времени кластер состоит всего из одной частицы ("зерна" роста). В дальнейшим рост кластера происходит за счет последовательного присоединения новых частиц в узлах по его периметру.

#### Структура кластеров Идена

Зависимость количества частиц в кластере и его радиусом гирации $R_g$ описывается соотношением $N \sim R_g^d$, где величина $d$ практически совпадает с евклидовой размерностью.

Радиус гирации определяется, как $R_g = \dfrac{1}{N}\sqrt{\sum\limits_{i=1}^{N} (\vec{r}_i - \vec{r})^2}$, где $\vec{r} = \dfrac{1}{N}\sum\limits_{i=1}^{N} \vec{r}_i$ соответствует координатам центра масс кластера.

Поверхность кластеров Идена имеет фрактальные свойства, а число узлов на периметре растет как $N_s \sim R_g^{d_f}$, где $d_f > 1$ $-$ фрактальная размерность периметра

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
class IdenClaster():
    """Данный класс позволяет создать кластер базовой модели Идена."""
    
    def __init__(self, lattice_size):
        self.lattice_size = int(lattice_size)
        self.l = int(self.lattice_size / 2)
        
        #Массив координат частиц
        self.particles = np.zeros((self.lattice_size, self.lattice_size))
        #"Зерно" роста
        self.particles[self.l][self.l] = 1
        
        #Таблица сдвига
        self.dx = np.array([1, 0, -1, 0], dtype=int)
        self.dy = np.array([0, -1, 0, 1], dtype=int)
        
        #Узлы "зерна" роста
        self.perimeter_x = self.dx + self.l
        self.perimeter_y = self.dy + self.l
        
        #Необходимы для записи временных изменений
        self.time = []
        self.x_time = []
        self.y_time = []
        self.type = []
        
        self.__time_recording(0)
        
        
    def growth(self):
        time = 0
        while abs(self.l - np.max(self.perimeter_x)) < self.l and abs(self.l - np.max(self.perimeter_y)) < self.l:
            
            self.__time_recording(time)
            
            #Выбор случайного узла частицы
            i = np.random.randint(0, len(self.perimeter_x))
            #Присоединение частицы
            self.particles[self.perimeter_x[i]][self.perimeter_y[i]] = 1
            
            #Вычисление нового периметра кластера
            for dx, dy in zip(self.dx, self.dy):
                x = self.perimeter_x[i] + dx
                y = self.perimeter_y[i] + dy
                
                if self.particles[x][y] == 0:
                    self.perimeter_x = np.append(self.perimeter_x, x)
                    self.perimeter_y = np.append(self.perimeter_y, y)
                    
            self.perimeter_x = np.delete(self.perimeter_x, i)
            self.perimeter_y = np.delete(self.perimeter_y, i)
            
            time += 1
            
        self.df = pd.DataFrame({
            'time': self.time, 
            'x': [x - self.l for x in self.x_time], 
            'y': [y - self.l for y in self.y_time], 
            'type' : self.type
        })
    
    def __time_recording(self, time):
        if time % 25 == 0:
                
            x_time, y_time = np.where(self.particles >= 0)
            self.x_time.extend(x_time)
            self.y_time.extend(y_time)
            self.time.extend([time] * len(x_time))
            
            for x, y in zip(x_time, y_time):
                if self.particles[x][y] == 0:
                    self.type.append(0)
                else:
                    self.type.append(1)
            
    def vizualization(self):
        px.defaults.width = 800
        px.defaults.height = 800
        fig = px.scatter(self.df, x='x', y='y', animation_frame='time', size="type", 
                         color=-np.sqrt(self.df['x']**2 + self.df['y']**2), range_x=[-self.l, self.l], 
                         range_y=[-self.l, self.l], size_max=10)
        
        fig.show()

In [11]:
n = 27
my_claster = IdenClaster(n)
my_claster.growth()

In [12]:
my_claster.vizualization()